## Installation
-  Run this only once

In [1]:
!git clone  https://github.com/csekankan/MTP-Recommender.git

Cloning into 'MTP-Recommender'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 130 (delta 69), reused 81 (delta 34), pack-reused 0
Receiving objects: 100% (130/130), 580.29 KiB | 2.90 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [4]:
!cp "/content/MTP-Recommender/collaborative filtering/graph based/param/param_gcn.yaml" "/content/param_gcn.yaml"

In [5]:
!pip install gdown
!pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.7/402.7 KB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.4 MB/s eta 0:00:00
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=917949 sha256=9c58486b28f5da216e5cce573cf0f0c89ad5265be7bfa7443a9d2b96612ad826
  Stored in directory: /root/.cache/pip/wheels/ec/bb/51/9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366433 sha256=617afac17ed540ffab788c679b52707bdf08e3c3ebc09287e8ee2a2fef3bdb5b
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built lightfm scikit-surprise

In [6]:
!pip install recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Data Loading

In [7]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np
from datetime import datetime

In [8]:
SEED=42

In [9]:
# !gdown https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct
# !unzip -o "data.zip"  -d  "/content"

# movie_10k_df = pd.read_csv("movies_min.csv")
# amazon_df =pd.read_csv('amazon_min.csv')
# book_df=pd.read_csv("book_min.csv")
# food_df = pd.read_csv('food_min.csv')
# ecommerce_df=pd.read_csv('ecom_min.csv')
# ciao_df=pd.read_csv('ciao_min.csv')

In [10]:
TIMESTAMP=None

In [11]:
!gdown https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct

Downloading...
From: https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct
To: /content/data.zip
100% 4.16M/4.16M [00:00<00:00, 187MB/s]


In [12]:
!unzip -o "data.zip"  -d  "/content"

Archive:  data.zip
  inflating: /content/amazon_min.csv  
  inflating: /content/book_min.csv   
  inflating: /content/ciao_min.csv   
  inflating: /content/ecom_min.csv   
  inflating: /content/food_min.csv   
  inflating: /content/movies_min.csv  


## Name of the file where user item interaction data avaiable?
- Column name should be in following name and order
- [ **itemID, userID , rating , timestamp** ]
- No issue if there is no column for **timestamp**

In [13]:
filename="movies_min.csv" #enter your filename with file path

In [14]:
rating_df = pd.read_csv(filename)


### Run below cell

In [15]:
import time 
if  'timestamp' in rating_df.columns:
  rating_df =rating_df[[ 'userID', 'itemID', 'rating','timestamp']]
  rating_df.columns =[ 'userID', 'itemID', 'rating','timestamp']
else:

  obj = time.gmtime(0)
  epoch = time.asctime(obj)
  curr_time = round(time.time()*1000)
  rating_df['timestamp']=curr_time
  rating_df =rating_df[[ 'userID', 'itemID', 'rating','timestamp']]
  rating_df.columns =[ 'userID', 'itemID', 'rating','timestamp']

In [16]:
NUM_OF_COLS=rating_df.shape[1]


In [17]:
u_users=list(rating_df.userID.unique())

In [18]:
u_items=list(rating_df.itemID.unique())

##### Code Setup

In [19]:
from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

In [20]:
from datetime import datetime


In [21]:
SEED=0
# top k items to recommend
TOP_K = 10

# Model parameters
EPOCHS = 20
BATCH_SIZE = 256

In [22]:
def runLighGCN(df_input):
  gcn_df=df_input.copy()
  ct = datetime.now()

  ts = ct.timestamp()
  
  gcn_df.reset_index()
  gcn_df=gcn_df[['userID','itemID','rating','timestamp']]
  gcn_item_ids = list(set(list(gcn_df.itemID.unique())))
  gcn_user_ids = list(set(list(gcn_df.userID.unique())))
  dict_items = {}
  index = 0
  for ids in sorted(gcn_item_ids):
      dict_items[ids] = index
      index += 1
  dict_users = {}
  index = 0
  for ids in sorted(gcn_user_ids):
      dict_users[ids] = index
      index += 1
  gcn_df["itemID"] = gcn_df["itemID"].map(dict_items)
  gcn_df["userID"] = gcn_df["userID"].map(dict_users)
  gcn_df.reset_index()
  train, test = python_stratified_split(gcn_df, ratio=0.8)
  data = ImplicitCF(train=train, test=test, seed=SEED)
  yaml_file = "param_gcn.yaml"
  hparams = prepare_hparams(yaml_file,
                            n_layers=4,
                            batch_size=BATCH_SIZE,
                            epochs=EPOCHS,
                            learning_rate=0.005,
                            eval_epoch=5,
                            top_k=TOP_K,
                           )
  model = LightGCN(hparams, data, seed=SEED)
  before= datetime.now()
  model.fit()
  after= datetime.now()
  duration=after-before
  topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)
  eval_map = map_at_k(test, topk_scores, k=TOP_K)
  eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
  eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
  eval_recall = recall_at_k(test, topk_scores, k=TOP_K)
  # Record results with papermill for tests
  print("map"+ str(eval_map))
  print("ndcg"+ str(eval_ndcg))
  print("precision"+ str( eval_precision))
  print("recall"+ str(eval_recall))
  return eval_map,eval_ndcg,eval_precision,eval_recall,topk_scores

## Run LightGCN

In [23]:


gcn_eval_map,gcn_eval_ndcg,gcn_eval_precision,gcn_eval_recall,preds=runLighGCN(rating_df)


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)2.4s: train loss = 0.44002 = (mf)0.43946 + (embed)0.00056
Epoch 2 (train)1.8s: train loss = 0.33415 = (mf)0.33296 + (embed)0.00120
Epoch 3 (train)1.7s: train loss = 0.30612 = (mf)0.30456 + (embed)0.00157
Epoch 4 (train)1.7s: train loss = 0.27628 = (mf)0.27425 + (embed)0.00203
Epoch 5 (train)1.7s + (eval)2.3s: train loss = 0.25635 = (mf)0.25382 + (embed)0.00253, recall = 0.19657, ndcg = 0.36141, precision = 0.30255, map = 0.11516
Epoch 6 (train)1.9s: train loss = 0.24823 = (mf)0.24529 + (embed)0.00294
Epoch 7 (train)1.9s: train loss = 0.24138 = (mf)0.23813 + (embed)0.00325
Epoch 8 (train)1.8s: train loss = 0.23261 = (mf)0.22908 + (embed)0.00353
Epoch 9 (train)1.7s: train loss = 0.22517 = (mf)0.22133 + (embed)0.00383
Epoch 10 (train)1.7s + (eval)0.1s: train loss = 0.22156 = (mf)0.21744 + (embed)0.00412, recall = 0.21516, ndcg = 0.38501, precision = 0.32439, map = 0.12721
Epoch

#### Recommendations

In [24]:
preds.head(5)

,userID,itemID,prediction
0,0,92,5.235608
1,0,161,4.426775
2,0,156,4.382618
3,0,192,4.350492
4,0,110,4.241471
